# Orchestration Testing

## Prerequisites

1. Kubernetes cluster runnig
2. The HPA metrics server is installed Kuber
    
    Check if the metrics server is running: 
    
    ```bash
    kubectl get apiservices | grep metrics
    ```

    If it is not isntalled, you can deploy it with this command: 

    ```bash
    kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
    ```

In [1]:
import sys
import os
import time

from kubernetes import client, config

## Import our system

In [2]:
sys.path.append ('..')
import utils
utils.add_to_sys_path(os.path.join('..', '..', 'scale'))

from orchestration import OrchestrationClient, KubernetesClient

### Kubernetes setup

In [3]:
namespace = 'cash-flow'
test_service = 'registration'

k8s_client = KubernetesClient(namespace=namespace)

# to get the pods
v1 = client.CoreV1Api()

### Utilities

In [4]:
def list_pods(namespace):
    print(f"Listing pods in the {namespace} namespace:")
    pods = v1.list_namespaced_pod(namespace=namespace)
    for pod in pods.items:
        print(pod.metadata.name)



### Get the current replicas for service

In [5]:

replicas = k8s_client.get_current_scale(test_service)

print(f"Current number of {test_service} replicas: {replicas}")

Current number of registration replicas: 1


### Scale up

In [6]:
initial_replicas = k8s_client.get_current_scale(test_service)

# scale up by 1 replica
k8s_client.scale_service('registration', 1) 


for _ in range(10):
    new_replicas = k8s_client.get_current_scale(test_service)
    if new_replicas > initial_replicas:
        print(f"Scale up successful. New number of {test_service} replicas: {new_replicas}")
        break
    else:
        time.sleep(1)

list_pods(namespace)

Scale up successful. New number of registration replicas: 2
Listing pods in the cash-flow namespace:
auth-5fd84d4f95-szmpr
celery-worker-7c98bc497f-8mnln
dms-57946db4f6-bwwr9
redis-6664db5d7c-dlffh
registration-677864c8bf-92t4d
registration-677864c8bf-nkfwx
tasking-5c96756555-ssxkh
transaction-647c77f96f-6w7t6
tx-db-dep-5cb5b7b4d6-k9g2j
user-db-dep-76f5ff8cc7-5btc9


### Scale down

In [7]:
initial_replicas = k8s_client.get_current_scale(test_service)

# scale down by 1 replica
k8s_client.scale_service('registration', -1)

for _ in range(10):
    new_replicas = k8s_client.get_current_scale(test_service)
    if new_replicas < initial_replicas:
        print(f"Scale down successful. New number of {test_service} replicas: {new_replicas}")
        break
    else:
        time.sleep(1)

# takes a couple seconds for the pod to be deleted
time.sleep(2)
        
list_pods(namespace)

Scale down successful. New number of registration replicas: 1
Listing pods in the cash-flow namespace:
auth-5fd84d4f95-szmpr
celery-worker-7c98bc497f-8mnln
dms-57946db4f6-bwwr9
redis-6664db5d7c-dlffh
registration-677864c8bf-92t4d
tasking-5c96756555-ssxkh
transaction-647c77f96f-6w7t6
tx-db-dep-5cb5b7b4d6-k9g2j
user-db-dep-76f5ff8cc7-5btc9
